<a href="https://colab.research.google.com/github/arshhad45/EDA/blob/main/Brain_Tumor_MRI_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    -



##### **Project Type**    - EDA/Regression/Classification/Unsupervised
##### **Contribution**    - Individual/Team
##### **Team Member 1 -** - Syed Arshad A

# **Project Summary -**

This project focused on building a deep learning-based image classification system to detect and classify brain tumors using MRI scans. Leveraging both a custom Convolutional Neural Network (CNN) and transfer learning with pretrained models (e.g., ResNet50, EfficientNetB0), the solution classifies MRI images into multiple tumor categories. The system was deployed via a Streamlit web app for real-time predictions.

Key components of the project include thorough data preprocessing (normalization, resizing), extensive data augmentation to improve generalization, and the use of callbacks like EarlyStopping and ModelCheckpoint during model training. Comprehensive evaluation metrics (accuracy, precision, recall, F1-score, confusion matrix) were used to compare model performance and select the most effective model for deployment.

The project also demonstrates practical use cases in AI-assisted medical diagnosis, early detection, research support, and telemedicine integration, making it highly relevant for real-world healthcare applications.

# **GitHub Link -**

https://github.com/arshhad45

# **Problem Statement**


Develop a deep learning-based model to accurately classify brain MRI images into different tumor types, improving early diagnosis and assisting radiologists with fast, reliable predictions. The solution should also include a user-friendly web interface for real-time image-based predictions.



# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Basic
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Image Processing
from PIL import Image
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


# Model Evaluation
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import itertools

# Visualization
import matplotlib.pyplot as plt

# Deployment - Streamlit
import streamlit as st

# Warnings
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
print(tf.__version__)



### Dataset Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

dataset_path = '/content/drive/MyDrive/Tumour'
print(os.listdir(dataset_path))


In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_path = "/content/drive/MyDrive/Tumour/train"
val_path = "/content/drive/MyDrive/Tumour/valid"

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


CNN Model

In [ ]:
def build_custom_cnn(input_shape=(224, 224, 3), num_classes=4):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model


In [ ]:
#Train Custom CNN
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('custom_cnn_model.h5', save_best_only=True)
]

# Build the custom CNN model
cnn_model = build_custom_cnn(input_shape=IMAGE_SIZE + (3,), num_classes=train_generator.num_classes)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_cnn = cnn_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)

In [ ]:
#Transfer Learning Model (ResNet50)

def build_transfer_model(base_model, preprocess_func, input_shape=(224, 224, 3), num_classes=4):
    base = base_model(weights='imagenet', include_top=False, input_shape=input_shape)
    base.trainable = False  # freeze base

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=output)
    return model


In [ ]:
#Build & Train with ResNet50:
resnet_model = build_transfer_model(ResNet50, resnet_preprocess)
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('resnet_model.h5', save_best_only=True)
]

history_resnet = resnet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)


In [ ]:
# Model Evaluation
# Evaluate on validation data
val_loss, val_acc = resnet_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc:.2f}")

# Confusion matrix
Y_pred = resnet_model.predict(val_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = val_generator.classes

print(classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys()))


In [ ]:
#Save Trained Models
cnn_model.save('custom_cnn_model.h5', save_format='h5')
resnet_model.save("resnet50_model.h5")


In [ ]:
#Compare Models
def plot_history(history, title):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'{title} - Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{title} - Loss')
    plt.legend()
    plt.show()

plot_history(history_cnn, "Custom CNN")
plot_history(history_resnet, "ResNet50")


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Set page config
st.set_page_config(page_title="Brain Tumor MRI Classifier", layout="centered")

# Load model
model = load_model("custom_cnn_model.h5")  # or "resnet50_model.h5"
class_names = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']

# Title and description
st.title("🧠 Brain Tumor MRI Classification")
st.write("Upload an MRI brain scan to predict the tumor type.")

# File upload
uploaded_file = st.file_uploader("Choose an MRI image", type=["jpg", "jpeg", "png"])

# Preprocessing
def preprocess_image(image, target_size=(224, 224)):
    image = image.resize(target_size)
    img_array = img_to_array(image)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Predict
if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption='Uploaded Image', use_column_width=True)

    st.write("⏳ Classifying...")
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    predicted_class = class_names[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    st.success(f"🔍 Prediction: **{predicted_class}**")
    st.info(f"📊 Confidence: **{confidence:.2f}%**")

In [ ]:
from pyngrok import ngrok
from google.colab import userdata

NGROK_AUTH_TOKEN = userdata.get('ngrok')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [ ]:
# Kill any previous tunnels if open
!pkill streamlit
!pkill ngrok
import time
time.sleep(2) # Add a small delay

# Start the streamlit app in the background
!nohup streamlit run app.py &

# Wait a few seconds and expose the port
# Use the recommended way to specify the port
public_url = ngrok.connect(addr=8501, proto='http')
print("Streamlit app is live at:", public_url)